In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Artificial Intelligence\n",
    "# 464\n",
    "# Project #5 - Image Classification\n",
    "\n",
    "## Before You Begin...\n",
    "00. We're using a Jupyter Notebook environment (tutorial available here: https://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html),\n",
    "01. Read the entire notebook before beginning your work, and\n",
    "02. Check the submission deadline on Gradescope.\n",
    "\n",
    "\n",
    "## General Directions for this Assignment\n",
    "00. Output format should be exactly as requested,\n",
    "01. Functions should do only one thing.\n",
    "\n",
    "\n",
    "## Before You Submit...\n",
    "00. Re-read the general instructions provided above, and\n",
    "01. Hit \"Kernel\"->\"Restart & Run All\". The first cell that is run should show [1], the second should show [2], and so on...\n",
    "02. Submit your notebook (as .ipynb, not PDF) using Gradescope, and\n",
    "03. Do not submit any other files."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Neural Networks: Image Classification\n",
    "\n",
    "For this assignment, we will explore Neural Networks for image classification using the MNIST dataset, which consists of handwritten digits (0-9). The goal is to classify these images correctly. We'll use PyTorch to explore different model architectures and compare their performance.\n",
    "\n",
    "More specifically: \n",
    "\n",
    "* Try three different models (varying in complexity),\n",
    "* Use cross validation to compare the models,\n",
    "* Report the winning model's performance on test data,\n",
    "* Include a table comparing the three models,\n",
    "* Document your process: what the results were, how the winning model was determined, loss function, activation, etc.\n",
    "\n",
    "Information on cross validation: https://scikit-learn.org/stable/modules/cross_validation.html (notice how final evaluation is the only time test data is used). No other directions for this assignment other than what's here and in the \"General Directions\" section. You have a lot of freedom with this assignment. Don't get carried away. It is expected the results may vary, being better or worse, due to the limitations of the dataset. Graders are not going to run your notebooks. The notebook will be read as a report on how different models were explored. Since you'll be using libraries, the emphasis will be on your ability to communicate your findings."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import torch\n",
    "import torch.nn as nn\n",
    "import torch.optim as optim\n",
    "from torch.utils.data import DataLoader\n",
    "from torchvision import datasets, transforms\n",
    "import numpy as np\n",
    "from sklearn.model_selection import KFold\n",
    "from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load MNIST dataset\n",
    "transform = transforms.Compose([\n",
    "    transforms.ToTensor(),\n",
    "    transforms.Normalize((0.1307,), (0.3081,))\n",
    "])\n",
    "\n",
    "train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)\n",
    "test_dataset = datasets.MNIST('./data', train=False, transform=transform)\n",
    "\n",
    "# Split training data into train and validation sets\n",
    "train_size = int(0.85 * len(train_dataset))\n",
    "val_size = len(train_dataset) - train_size\n",
    "train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])\n",
    "\n",
    "# Create data loaders\n",
    "train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)\n",
    "val_loader = DataLoader(val_dataset, batch_size=64)\n",
    "test_loader = DataLoader(test_dataset, batch_size=64)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Define three different models\n",
    "class SimpleCNN(nn.Module):\n",
    "    def __init__(self):\n",
    "        super(SimpleCNN, self).__init__()\n",
    "        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)\n",
    "        self.relu = nn.ReLU()\n",
    "        self.maxpool = nn.MaxPool2d(2)\n",
    "        self.fc1 = nn.Linear(5408, 128)\n",
    "        self.fc2 = nn.Linear(128, 10)\n",
    "        \n",
    "    def forward(self, x):\n",
    "        x = self.conv1(x)\n",
    "        x = self.relu(x)\n",
    "        x = self.maxpool(x)\n",
    "        x = x.view(x.size(0), -1)\n",
    "        x = self.fc1(x)\n",
    "        x = self.relu(x)\n",
    "        x = self.fc2(x)\n",
    "        return x\n",
    "\n",
    "class MediumCNN(nn.Module):\n",
    "    def __init__(self):\n",
    "        super(MediumCNN, self).__init__()\n",
    "        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)\n",
    "        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)\n",
    "        self.relu = nn.ReLU()\n",
    "        self.maxpool = nn.MaxPool2d(2)\n",
    "        self.dropout = nn.Dropout(0.25)\n",
    "        self.fc1 = nn.Linear(1600, 128)\n",
    "        self.fc2 = nn.Linear(128, 10)\n",
    "        \n",
    "    def forward(self, x):\n",
    "        x = self.conv1(x)\n",
    "        x = self.relu(x)\n",
    "        x = self.conv2(x)\n",
    "        x = self.relu(x)\n",
    "        x = self.maxpool(x)\n",
    "        x = self.dropout(x)\n",
    "        x = x.view(x.size(0), -1)\n",
    "        x = self.fc1(x)\n",
    "        x = self.relu(x)\n",
    "        x = self.dropout(x)\n",
    "        x = self.fc2(x)\n",
    "        return x\n",
    "\n",
    "class ComplexCNN(nn.Module):\n",
    "    def __init__(self):\n",
    "        super(ComplexCNN, self).__init__()\n",
    "        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)\n",
    "        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)\n",
    "        self.conv3 = nn.Conv2d(64, 128, kernel_size=3)\n",
    "        self.relu = nn.ReLU()\n",
    "        self.maxpool = nn.MaxPool2d(2)\n",
    "        self.dropout = nn.Dropout(0.25)\n",
    "        self.bn1 = nn.BatchNorm2d(32)\n",
    "        self.bn2 = nn.BatchNorm2d(64)\n",
    "        self.bn3 = nn.BatchNorm2d(128)\n",
    "        self.fc1 = nn.Linear(128, 64)\n",
    "        self.fc2 = nn.Linear(64, 10)\n",
    "        \n",
    "    def forward(self, x):\n",
    "        x = self.conv1(x)\n",
    "        x = self.bn1(x)\n",
    "        x = self.relu(x)\n",
    "        x = self.conv2(x)\n",
    "        x = self.bn2(x)\n",
    "        x = self.relu(x)\n",
    "        x = self.maxpool(x)\n",
    "        x = self.conv3(x)\n",
    "        x = self.bn3(x)\n",
    "        x = self.relu(x)\n",
    "        x = self.maxpool(x)\n",
    "        x = self.dropout(x)\n",
    "        x = x.view(x.size(0), -1)\n",
    "        x = self.fc1(x)\n",
    "        x = self.relu(x)\n",
    "        x = self.dropout(x)\n",
    "        x = self.fc2(x)\n",
    "        return x"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):\n",
    "    train_losses = []\n",
    "    val_losses = []\n",
    "    train_accs = []\n",
    "    val_accs = []\n",
    "    \n",
    "    for epoch in range(num_epochs):\n",
    "        # Training phase\n",
    "        model.train()\n",
    "        running_loss = 0.0\n",
    "        correct = 0\n",
    "        total = 0\n",
    "        \n",
    "        for inputs, labels in train_loader:\n",
    "            optimizer.zero_grad()\n",
    "            outputs = model(inputs)\n",
    "            loss = criterion(outputs, labels)\n",
    "            loss.backward()\n",
    "            optimizer.step()\n",
    "            \n",
    "            running_loss += loss.item()\n",
    "            _, predicted = torch.max(outputs.data, 1)\n",
    "            total += labels.size(0)\n",
    "            correct += (predicted == labels).sum().item()\n",
    "        \n",
    "        train_loss = running_loss / len(train_loader)\n",
    "        train_acc = 100 * correct / total\n",
    "        train_losses.append(train_loss)\n",
    "        train_accs.append(train_acc)\n",
    "        \n",
    "        # Validation phase\n",
    "        model.eval()\n",
    "        val_loss = 0.0\n",
    "        correct = 0\n",
    "        total = 0\n",
    "        \n",
    "        with torch.no_grad():\n",
    "            for inputs, labels in val_loader:\n",
    "                outputs = model(inputs)\n",
    "                loss = criterion(outputs, labels)\n",
    "                val_loss += loss.item()\n",
    "                _, predicted = torch.max(outputs.data, 1)\n",
    "                total += labels.size(0)\n",
    "                correct += (predicted == labels).sum().item()\n",
    "        \n",
    "        val_loss = val_loss / len(val_loader)\n",
    "        val_acc = 100 * correct / total\n",
    "        val_losses.append(val_loss)\n",
    "        val_accs.append(val_acc)\n",
    "        \n",
    "        print(f'Epoch {epoch+1}/{num_epochs}:')\n",
    "        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')\n",
    "        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')\n",
    "        print('-' * 50)\n",
    "    \n",
    "    return train_losses, val_losses, train_accs, val_accs"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def evaluate_model(model, test_loader):\n",
    "    model.eval()\n",
    "    all_preds = []\n",
    "    all_labels = []\n",
    "    \n",
    "    with torch.no_grad():\n",
    "        for inputs, labels in test_loader:\n",
    "            outputs = model(inputs)\n",
    "            _, predicted = torch.max(outputs.data, 1)\n",
    "            all_preds.extend(predicted.numpy())\n",
    "            all_labels.extend(labels.numpy())\n",
    "    \n",
    "    accuracy = accuracy_score(all_labels, all_preds)\n",
    "    precision = precision_score(all_labels, all_preds, average='weighted')\n",
    "    recall = recall_score(all_labels, all_preds, average='weighted')\n",
    "    f1 = f1_score(all_labels, all_preds, average='weighted')\n",
    "    \n",
    "    return accuracy, precision, recall, f1"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Train and evaluate all three models\n",
    "models = {\n",
    "    'Simple CNN': SimpleCNN(),\n",
    "    'Medium CNN': MediumCNN(),\n",
    "    'Complex CNN': ComplexCNN()\n",
    "}\n",
    "\n",
    "results = {}\n",
    "\n",
    "for name, model in models.items():\n",
    "    print(f'\\nTraining {name}...')\n",
    "    criterion = nn.CrossEntropyLoss()\n",
    "    optimizer = optim.Adam(model.parameters(), lr=0.001)\n",
    "    \n",
    "    train_losses, val_losses, train_accs, val_accs = train_model(\n",
    "        model, train_loader, val_loader, criterion, optimizer, num_epochs=10\n",
    "    )\n",
    "    \n",
    "    accuracy, precision, recall, f1 = evaluate_model(model, test_loader)\n",
    "    \n",
    "    results[name] = {\n",
    "        'accuracy': accuracy,\n",
    "        'precision': precision,\n",
    "        'recall': recall,\n",
    "        'f1': f1,\n",
    "        'train_losses': train_losses,\n",
    "        'val_losses': val_losses,\n",
    "        'train_accs': train_accs,\n",
    "        'val_accs': val_accs\n",
    "    }"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create comparison table\n",
    "import pandas as pd\n",
    "\n",
    "comparison_data = {\n",
    "    'Model': [],\n",
    "    'Accuracy': [],\n",
    "    'Precision': [],\n",
    "    'Recall': [],\n",
    "    'F1 Score': []\n",
    "}\n",
    "\n",
    "for name, metrics in results.items():\n",
    "    comparison_data['Model'].append(name)\n",
    "    comparison_data['Accuracy'].append(f'{metrics[\"accuracy\"]:.4f}')\n",
    "    comparison_data['Precision'].append(f'{metrics[\"precision\"]:.4f}')\n",
    "    comparison_data['Recall'].append(f'{metrics[\"recall\"]:.4f}')\n",
    "    comparison_data['F1 Score'].append(f'{metrics[\"f1\"]:.4f}')\n",
    "\n",
    "comparison_df = pd.DataFrame(comparison_data)\n",
    "print('\\nModel Comparison:')\n",
    "print(comparison_df.to_string(index=False))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Plot training and validation metrics\n",
    "def plot_metrics(results):\n",
    "    fig, axes = plt.subplots(2, 2, figsize=(15, 10))\n",
    "    fig.suptitle('Model Performance Comparison')\n",
    "    \n",
    "    # Plot training loss\n",
    "    for name, metrics in results.items():\n",
    "        axes[0, 0].plot(metrics['train_losses'], label=name)\n",
    "    axes[0, 0].set_title('Training Loss')\n",
    "    axes[0, 0].set_xlabel('Epoch')\n",
    "    axes[0, 0].set_ylabel('Loss')\n",
    "    axes[0, 0].legend()\n",
    "    \n",
    "    # Plot validation loss\n",
    "    for name, metrics in results.items():\n",
    "        axes[0, 1].plot(metrics['val_losses'], label=name)\n",
    "    axes[0, 1].set_title('Validation Loss')\n",
    "    axes[0, 1].set_xlabel('Epoch')\n",
    "    axes[0, 1].set_ylabel('Loss')\n",
    "    axes[0, 1].legend()\n",
    "    \n",
    "    # Plot training accuracy\n",
    "    for name, metrics in results.items():\n",
    "        axes[1, 0].plot(metrics['train_accs'], label=name)\n",
    "    axes[1, 0].set_title('Training Accuracy')\n",
    "    axes[1, 0].set_xlabel('Epoch')\n",
    "    axes[1, 0].set_ylabel('Accuracy (%)')\n",
    "    axes[1, 0].legend()\n",
    "    \n",
    "    # Plot validation accuracy\n",
    "    for name, metrics in results.items():\n",
    "        axes[1, 1].plot(metrics['val_accs'], label=name)\n",
    "    axes[1, 1].set_title('Validation Accuracy')\n",
    "    axes[1, 1].set_xlabel('Epoch')\n",
    "    axes[1, 1].set_ylabel('Accuracy (%)')\n",
    "    axes[1, 1].legend()\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "\n",
    "plot_metrics(results)"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}